In [ ]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Optimizing Joins")
    .master("spark://spark-master:7077")
    .config("spark.cores.max", 16)
    .config("spark.executor.cores", 4)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/04 16:46:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Disable AQE and Broadcast join

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

#### Join Big and Small table - SortMerge vs BroadCast Join

In [ ]:
# Read EMP CSV data

_schema = "first_name string, last_name string, job_title string, dob string, email string, phone string, salary double, department_id int"

emp = (
    spark.read.format("csv")
    .schema(_schema)
    .option("header", True)
    .load("hdfs://namenode:9000/input/data/employee_records.csv")
)

In [ ]:
# Read DEPT CSV data

_dept_schema = "department_id int, department_name string, description string, city string, state string, country string"

dept = (
    spark.read.format("csv")
    .schema(_dept_schema)
    .option("header", True)
    .load("hdfs://namenode:9000/input/data/department_data.csv")
)

In [5]:
emp.count()

1000000

In [6]:
dept.count()

10

In [ ]:
# Join Datasets
from pyspark.sql import functions as F

df_joined = emp.join(
    F.broadcast(dept), on=emp.department_id == dept.department_id, how="left_outer"
)

In [8]:
df_joined.write.format("noop").mode("overwrite").save()

In [9]:
df_joined.explain()

== Physical Plan ==
*(2) BroadcastHashJoin [department_id#7], [department_id#16], LeftOuter, BuildRight, false
:- FileScan csv [first_name#0,last_name#1,job_title#2,dob#3,email#4,phone#5,salary#6,department_id#7] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://namenode:9000/input/data/employee_records.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<first_name:string,last_name:string,job_title:string,dob:string,email:string,phone:string,s...
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=109]
   +- *(1) Filter isnotnull(department_id#16)
      +- FileScan csv [department_id#16,department_name#17,description#18,city#19,state#20,country#21] Batched: false, DataFilters: [isnotnull(department_id#16)], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://namenode:9000/input/data/department_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(department_id)], Rea

In [ ]:
# This is what happens without Broadcast
df_joined_no_bc = emp.join(
    dept, on=emp.department_id == dept.department_id, how="left_outer"
)
df_joined_no_bc.write.format("noop").mode("overwrite").save()

In [ ]:
# This is what happens when we Broadcast a very large dataframe
emp = emp.coalesce(1)  # 1 huge partition
df_joined_no_bc = dept.join(
    F.broadcast(emp), on=emp.department_id == dept.department_id, how="left_outer"
)
df_joined_no_bc.write.format("noop").mode("overwrite").save()

24/11/04 16:53:50 WARN TaskSetManager: Lost task 0.0 in stage 9.0 (TID 229) (172.18.0.8 executor 1): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$1227/0x00000008409ef040.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1849)
	at java.base/java.io.ObjectOutputStream.write(ObjectOut

Py4JJavaError: An error occurred while calling o82.save.
: java.util.concurrent.ExecutionException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 4 times, most recent failure: Lost task 0.4 in stage 9.0 (TID 233) (172.18.0.6 executor 3): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$953/0x00000008408ea040.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByte

#### Join Big and Big table - SortMerge without Buckets

In [12]:
# Read Sales data

sales = spark.read.parquet("hdfs://namenode:9000/input/data/sales.parquet")

In [14]:
sales.count()

1102575

In [ ]:
# Read City data

city_schema = (
    "city_id string, city string, state string, state_abv string, country string"
)

city = (
    spark.read.format("csv")
    .schema(city_schema)
    .option("header", True)
    .load("hdfs://namenode:9000/input/data/cities.csv")
)

In [16]:
city.count()

2349391

In [ ]:
# Join Data

df_sales_joined = sales.join(city, on=sales.city_id == city.city_id, how="left_outer")

In [18]:
df_sales_joined.write.format("noop").mode("overwrite").save()

In [31]:
# Partitions deep dive

print(sales.rdd.getNumPartitions())
print(city.rdd.getNumPartitions())
print(df_sales_joined.rdd.getNumPartitions())

8
8
200


In [ ]:
sales.withColumn("partition_id", F.spark_partition_id()).groupby(
    "partition_id"
).count().show()

+------------+-------+
|partition_id|  count|
+------------+-------+
|           6|  53999|
|           3|1048576|
+------------+-------+



In [39]:
sales.select("trx_id").distinct().count()

1101905

In [44]:
sales.select("city_id").distinct().count()

245

In [53]:
sales.groupby("city_id").count().orderBy("count", ascending=False).show()

+----------+-----+
|   city_id|count|
+----------+-----+
|  45522086| 9796|
| 856233063| 9724|
| 781290085| 9715|
|1217211842| 9697|
|1607451095| 9662|
|2055198208| 9647|
|1678157838| 9641|
| 559832710| 9612|
| 485114748| 9596|
| 831124332| 9586|
|  28424447| 9575|
|1717498102| 9573|
|1985376021| 9562|
|1141716004| 9555|
| 287177635| 9552|
|2096878759| 9546|
|1676567417| 9543|
|1606354386| 9538|
|2052535508| 9525|
| 275328977| 9522|
+----------+-----+
only showing top 20 rows



In [40]:
sales.count()

1102575

In [ ]:
sales.withColumn("partition_id", F.spark_partition_id()).groupby("city_id").agg(
    F.countDistinct("partition_id").alias("count")
).orderBy("count", ascending=False).show()

+----------+-----+
|   city_id|count|
+----------+-----+
| 198096267|    2|
| 293622227|    2|
|2096878759|    2|
| 985710244|    2|
|1606354386|    2|
|1678157838|    2|
| 407629665|    2|
| 930259917|    2|
| 220566264|    2|
| 632305754|    2|
|1683103551|    2|
|  45522086|    2|
| 350411713|    2|
|2078628066|    2|
| 414653088|    2|
|1840913006|    2|
| 971443623|    2|
| 381349011|    2|
|1802488818|    2|
|2033929143|    2|
+----------+-----+
only showing top 20 rows



In [ ]:
# REPARTITIONING OF CITY_ID COLUMN. NOW ALL RECORDS FOR THE SAME CITY_ID WILL BE AT THE SAME PARTITION
sales = sales.repartition(8, "city_id")
sales.withColumn("partition_id", F.spark_partition_id()).groupby("city_id").agg(
    F.countDistinct("partition_id").alias("count")
).orderBy("count", ascending=False).show()

+----------+-----+
|   city_id|count|
+----------+-----+
|1620965190|    1|
| 576817662|    1|
|1223420625|    1|
| 831124332|    1|
|2056066328|    1|
|1802488818|    1|
|1610115143|    1|
|1610133005|    1|
|1194163531|    1|
| 216135201|    1|
|1574873504|    1|
|  45522086|    1|
|1602735059|    1|
| 380143978|    1|
|1296036143|    1|
| 287177635|    1|
|  77397141|    1|
| 585903816|    1|
|1243655802|    1|
| 275328977|    1|
+----------+-----+
only showing top 20 rows



In [ ]:
city.withColumn("partition_id", F.spark_partition_id()).groupby(
    "partition_id"
).count().show()

+------------+------+
|partition_id| count|
+------------+------+
|           1|309558|
|           6|309876|
|           3|309622|
|           5|309730|
|           4|309817|
|           7|181567|
|           2|309607|
|           0|309614|
+------------+------+



##### Write Sales and City data in Buckets

In [ ]:
# Write Sales data in Buckets

sales.write.format("csv").mode("overwrite").bucketBy(4, "city_id").option(
    "header", True
).option("path", "hdfs://namenode:9000/input/data/sales_bucket").saveAsTable(
    "sales_bucket"
)

In [ ]:
# Write City data in Buckets

city.write.format("csv").mode("overwrite").bucketBy(4, "city_id").option(
    "header", True
).option("path", "hdfs://namenode:9000/input/data/city_bucket").saveAsTable(
    "city_bucket"
)

In [27]:
# Check tables

spark.sql("show tables in default").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default| city_bucket|      false|
|  default|sales_bucket|      false|
+---------+------------+-----------+



#### Join Sales and City data - SortMerge with Bucket

In [28]:
# Read Sales table

sales_bucket = spark.read.table("sales_bucket")

In [29]:
# Read City table

city_bucket = spark.read.table("city_bucket")

In [ ]:
# Join datasets

df_joined_bucket = city_bucket.join(
    sales_bucket, on=sales_bucket.city_id == city_bucket.city_id, how="left_outer"
)

In [33]:
# Write dataset

df_joined_bucket.write.format("noop").mode("overwrite").save()

In [70]:
df_joined_bucket.explain()

== Physical Plan ==
*(4) SortMergeJoin [city_id#1270L], [cast(city_id#1277 as bigint)], LeftOuter
:- *(1) Sort [city_id#1270L ASC NULLS FIRST], false, 0
:  +- FileScan csv spark_catalog.default.sales_bucket[transacted_at#1265,trx_id#1266L,retailer_id#1267L,description#1268,amount#1269,city_id#1270L] Batched: false, Bucketed: true, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://namenode:9000/input/data/sales_bucket], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<transacted_at:string,trx_id:bigint,retailer_id:bigint,description:string,amount:double,cit..., SelectedBucketsCount: 4 out of 4
+- *(3) Sort [cast(city_id#1277 as bigint) ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(cast(city_id#1277 as bigint), 4), ENSURE_REQUIREMENTS, [plan_id=2196]
      +- *(2) Filter isnotnull(city_id#1277)
         +- FileScan csv spark_catalog.default.city_bucket[city_id#1277,city#1278,state#1279,state_abv#1280,country#1281] Batched: false, Bucketed: 

In [34]:
spark.stop()